In [45]:
import pandas as pd
import math
import nltk

# Google Doc to Cleaned Stories

In [90]:
# read data
df_andrew = pd.read_excel("./stories.xlsx",sheet_name="Andrew",header=0,converters={"Prompt/Stories":str})
df_adi = pd.read_excel("./stories.xlsx",sheet_name="Adi",header=0,converters={"Prompt/Stories":str})
# adi
df_adi = df_adi.rename(index=str, columns={      "Unnamed: 3": "Group ID",
                                                 "Unnamed: 4": "Model",
                                                 "Unnamed: 6": "Story"})
adi_index = df_adi["Group ID"]%2==0
df_adi = df_adi.loc[adi_index]
# andrew
df_andrew = df_andrew.rename(index=str, columns={"Unnamed: 3": "Group ID",
                                                 "Unnamed: 4": "Model",
                                                 "Unnamed: 6": "Story"})
andrew_index = df_andrew["Group ID"]%2==1
df_andrew = df_andrew.loc[andrew_index]
# concatenate
df = pd.concat([df_adi,df_andrew])
df = df.drop(columns=["Unnamed: 10","Unnamed: 9","Unk","Even/Odd"])
#
numbers = []
prompts = []
for x in df["Prompt/Stories"]:
    try:
        num = float(x)
        prompts.append("")
        if(math.isnan(num)):
            numbers.append(0.0)
        else:
            numbers.append(num)
    except:
        prompts.append(x)
        numbers.append(0.0)
df = df.drop(columns=["Prompt/Stories"])
df["Prompts"] = prompts
df["Perplexity"] = numbers
# turn nan scores into 0s
scores = []
for x in df["Score"]:
    if(math.isnan(x)):
        scores.append(0.0)
    else:
        scores.append(x)
df["Score"] = scores
group_sums = df.groupby(['Group ID'])['Score'].sum()
# get stories that were scored by humans
complete = []
for index, row in df.iterrows():
    group = row["Group ID"]
    if(group_sums.get(group,0.0)!=6.0):
        complete.append(False)
    else:
        complete.append(True)
df = df.loc[complete]
# calculate perplexity with nltk
# write df to excel
df.to_excel("cleaned_stories.xlsx")

# Cleaned Stories with Ensemble to Complete Ranks

In [135]:
# read data
df = pd.read_excel("./stories_complete.xlsx",header=0)
df = df.drop(columns=["Unnamed: 8"])
df = df.rename(columns={"Unnamed: 9": "Ensemble", "Score": "Human Score"})
# ensemble ranks
df["Ensemble Rank"] = df.groupby("Group ID")["Ensemble"].rank(ascending=True).astype(int,errors="ignore")
# perplexity ranks
perplexity = []
for index, row in df.iterrows():
    if row["Model"]=="S":
        perplexity.append(float("nan"))
    else:
        perplexity.append(row["Perplexity"])
df["Perplexity"] = perplexity
df["Perplexity Rank"] = df.groupby("Group ID")["Perplexity"].rank(ascending=True).astype(int,errors="ignore")
# human scores
human = []
for index, row in df.iterrows():
    if row["Model"]=="S":
        human.append(float("nan"))
    else:
        human.append(row["Human Score"])
df["Human Score"] = human
# write xlsx
df.to_excel("complete_ranks.xlsx")

In [134]:
df.head(20)

,Count,Group ID,Log-Probability,Model,Human Score,Story,Prompts,Perplexity,Ensemble,Ensemble Rank,Perplexity Rank
0,0.0,0,0.944072,S,NaN,NaN,[ WP ] Leonardo DiCaprio in a fit of rage begi...,NaN,NaN,NaN,NaN
1,861.0,0,0.342777,H,2.0,I had n't been to a movie since I was a child ...,NaN,-1.901203,22.924102,1.0,1.0
2,3220.0,0,0.827215,T,3.0,The wet marble floor pressed on his cheek like...,NaN,0.000000,104.559351,3.0,3.0
3,937.0,0,0.913668,FusionH,1.0,"`` You know , we do n't really get to choose w...",NaN,-1.747556,24.071152,2.0,2.0
8,2416.0,2,0.931040,T,3.0,"George fled . Not terrifically fast , not at h...",NaN,0.000000,103.989352,3.0,3.0
9,1103.0,2,0.937600,H,1.0,"<unk> , <unk> , and <unk> <unk> , <newline> <n...",NaN,-0.410736,34.240087,2.0,2.0
10,0.0,2,0.810164,S,NaN,NaN,[ EU ] <unk> Bean has a hard time leaving his ...,NaN,NaN,NaN,NaN
11,898.0,2,0.382921,FusionH,2.0,`` The last man is in the room . He is not the...,NaN,-2.091001,25.331524,1.0,1.0
16,4726.0,4,0.589270,T,3.0,`` Hi dad ! '' <newline> <newline> The words s...,NaN,0.000000,93.088849,3.0,3.0
17,818.0,4,0.853060,FusionH,2.0,"I sat there for a moment , thinking about the ...",NaN,-1.826996,21.382103,1.0,2.0
